In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from Artha.nlp_extraction import run_pipeline, save_backup, load_backup, to_doc_dict
import Artha.data_process as dp
from Artha.neodb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import json
import numpy as np
np.set_printoptions(suppress=True)
from datetime import datetime, timedelta
from Artha.mentions import *
from collections import Counter

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
neo.session.run('''CREATE CONSTRAINT twitter_username IF NOT EXISTS ON (n:Person)
                ASSERT n.username IS UNIQUE''')
neo.session.run('''CREATE CONSTRAINT coin_name IF NOT EXISTS ON (n:Coin)
                ASSERT n.ticker IS UNIQUE''')


CPU times: user 2.1 s, sys: 276 ms, total: 2.37 s
Wall time: 3.28 s


In [2]:
follow_weight = .5
print(len(neo.get_nodes()), len(neo.get_relations()))
# without mentions 711 6958

105 206


In [3]:
neo.clear_db()

deleted database


In [4]:
%%time
# add follows between only people I follow, can change by getting rid of second 2 last line for anyone a person follows

# all_people = [i[1:-4] for i in os.listdir("../data/follows/")]
# checkra_people = [c[3] for c in dp.load_following("checkra_") ]


selected_people = ["BTC_JackSparrow", "razoreth", "Nostranomist", "CryptoKaleo", "nebraskangooner", "Rager", "SavageBTC"]
selected_nodes = [c for c in dp.load_following("checkra_") if c[3] in selected_people]

selected_follows = dp.follow_edges(selected_people, follow_weight)

CPU times: user 7.54 ms, sys: 1.98 ms, total: 9.52 ms
Wall time: 8.11 ms


In [5]:
%%time
# print(len(follows))
neo.clear_db()

neo.load_follow_nodes(selected_nodes)
neo.load_follow_nodes(selected_follows)
neo.load_follow_relations(selected_follows)

print(len(neo.get_nodes()), len(neo.get_relations()))


deleted database
loaded People
loaded People
loaded relations
7 15
CPU times: user 10.3 ms, sys: 3.86 ms, total: 14.2 ms
Wall time: 89.4 ms


In [6]:
%%time
# load all tweets
# all_tweets = []
# for ind, username in tqdm(enumerate(selected_people)):
#     cur_tweets = dp.load_tweets(username, location = "../data/tweets1/u")
#     all_tweets.extend([tweet for tweet in dp.clean_tweets(cur_tweets, username)])

# print("\n", len(all_tweets))
# #process all tweets at once
# docs = run_pipeline(all_tweets)
# 

docs = load_backup()
doc_dict = to_doc_dict(docs)

loaded
100%|██████████| 14474/14474 [00:00<00:00, 155544.00it/s]CPU times: user 5.62 s, sys: 147 ms, total: 5.76 s
Wall time: 5.77 s



In [7]:
date = "04/03/2021 00:00:00"
# code for multiple graphs over time

for i in tqdm(range(1)):
    date = from_datetime(to_datetime(date)+timedelta(days=i))
    all_mentions = []
    for username in selected_people:
        cur_mentions = get_mention_edges(doc_dict[username],
                                        username,
                                        follow_weight = follow_weight,
                                        win_start_date = date)
        all_mentions.extend(cur_mentions)
    print("\ndone", date)


100%|██████████| 1/1 [00:00<00:00,  2.69it/s]
done 04/03/2021 00:00:00



In [8]:
%%time
print(len(all_mentions))
neo.load_mentions(all_mentions)

191
CPU times: user 3.22 ms, sys: 2.19 ms, total: 5.42 ms
Wall time: 73.5 ms


[<Record count(r)=191>]

In [9]:
%%time

final_scores = neo.pagerank_scores(auto=True)
final_scores

CPU times: user 7.57 ms, sys: 1.96 ms, total: 9.54 ms
Wall time: 148 ms


[('BTC', 13.019977434724568),
 ('ETH', 2.4968253123573962),
 ('LTC', 1.2021665407810358),
 ('BTT', 1.0930827969219539),
 ('USD', 0.912968051037751),
 ('ENJ', 0.8219104532152415),
 ('TRX', 0.7271394041134047),
 ('BNB', 0.7195571100804957),
 ('UTK', 0.7036860298831015),
 ('UNI', 0.690762960142456),
 ('LINA', 0.5898421322228385),
 ('CRV', 0.49487193521345035),
 ('SCRT', 0.40850205213646407),
 ('HNST', 0.39773743371479214),
 ('FTT', 0.3652528362057638),
 ('DAI', 0.33993184982100505),
 ('AVAX', 0.32012211590772494),
 ('USDT', 0.31456115646287797),
 ('LINK', 0.3091509295860304),
 ('ETF', 0.30401939707226117),
 ('DOGE', 0.30197511042933917),
 ('USDC', 0.30007018170435906),
 ('SXP', 0.2923516809125431),
 ('XRP', 0.2906502604600973),
 ('SOL', 0.2866141424688976),
 ('KNC', 0.2847308282507584),
 ('SUSHI', 0.2780153848696501),
 ('MKR', 0.2716632260417099),
 ('COMP', 0.2654570596350823),
 ('SUN', 0.2637344300863333),
 ('1INCH', 0.26229711475898504),
 ('CND', 0.2602540055086137),
 ('WRX', 0.25808402

In [10]:
# make graphview that only looks at people who mention coins, and coins that are mentioned

# exchange tokens might have more mentions
# factor in mentions of coins in threads, so it gets more mentions than just 1